# Geospatial Data Import

In [1]:
# Libraries
import ee
import geemap
import geopandas as gpd
import json

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

True

In [4]:
# Initialize the library.
ee.Initialize()

In [51]:
# Test API: Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


Load the population grid

In [5]:
# Path to your GeoPackage file
gpkg_path = "/Users/laurianeteyssier/Projects/rubeho_mapper_tz/spatial_prep/onlypop_base_500m.gpkg"

# Read the polygon layer
gdf = gpd.read_file(gpkg_path)
print(gdf.head(10))
print(gdf.shape)

# Simplify the geometries to reduce complexity (optional)
gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.05, preserve_topology=True)

       grid_id                                           geometry
0  G_0000_1068  MULTIPOLYGON (((33.48466 -6.95182, 33.48465 -6...
1  G_0000_1069  MULTIPOLYGON (((33.48465 -6.9473, 33.48465 -6....
2  G_0001_1067  MULTIPOLYGON (((33.48919 -6.95634, 33.48918 -6...
3  G_0001_1068  MULTIPOLYGON (((33.48918 -6.95181, 33.48918 -6...
4  G_0001_1069  MULTIPOLYGON (((33.48918 -6.94729, 33.48917 -6...
5  G_0001_1070  MULTIPOLYGON (((33.48917 -6.94277, 33.48917 -6...
6  G_0002_1067  MULTIPOLYGON (((33.49371 -6.95633, 33.49371 -6...
7  G_0002_1068  MULTIPOLYGON (((33.49371 -6.95181, 33.4937 -6....
8  G_0002_1069  MULTIPOLYGON (((33.4937 -6.94729, 33.4937 -6.9...
9  G_0002_1070  MULTIPOLYGON (((33.4937 -6.94276, 33.49369 -6....
(837977, 2)


In [53]:
# Create the map
m = geemap.Map()

bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
lon = (bounds[0] + bounds[2]) / 2
lat = (bounds[1] + bounds[3]) / 2
m.set_center(lon, lat, zoom=8)

In [54]:
# Display in geemap (client-side visualization)
# geojson = json.loads(gdf.to_json())

# TODO optimize
# m.add_geojson(geojson, layer_name="Simplified Population Polygons")

# Display
# m

Geospatial embeddings

In [55]:
# Convert to Earth Engine geometry for use in GEE operations (run for about 30 seconds)
increment = 10000  # number of columns to process in each batch
print(gdf.shape)
for i in range(0, gdf.shape[0], increment):  # loop
    roi = geemap.geopandas_to_ee(gdf[i:i+increment])
    print(f"Processing rows {i} to {i+increment}...")

    dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL').filterDate('2024-01-01', '2025-01-01').filterBounds(roi).first()
    band_names = dataset.bandNames().getInfo()

    reduced = dataset.reduceRegions(
        collection=roi,
        reducer=ee.Reducer.mean(),
        scale=500  # adjust to your preferred spatial resolution (m)
    )

    selectors = ['grid_id'] + band_names

    # Export the image to a GeoTIFF file.
    task = ee.batch.Export.table.toDrive(
        collection=reduced,
        description='mean_satellite_embeddings_per_polygon',
        fileNamePrefix=f'polygon_embeddings_2025_columns_{i}_to_{i+increment}',
        fileFormat='CSV',
        selectors=selectors
    )

    task.start()
    print("✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.")

(837977, 2)
Processing rows 0 to 1000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 1000 to 2000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 2000 to 3000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 3000 to 4000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 4000 to 5000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 5000 to 6000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 6000 to 7000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 7000 to 8000...
✅ Export started to Google Drive. Check https://code.earthengin

In [56]:
tasks = ee.batch.Task.list()
for t in tasks:
    print(t.status())

{'state': 'READY', 'description': 'mean_satellite_embeddings_per_polygon', 'priority': 100, 'creation_timestamp_ms': 1759918272925, 'update_timestamp_ms': 1759918272925, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'FT33BSHZVUHD3OT5FVPINPHK', 'name': 'projects/52933960335/operations/FT33BSHZVUHD3OT5FVPINPHK'}
{'state': 'READY', 'description': 'mean_satellite_embeddings_per_polygon', 'priority': 100, 'creation_timestamp_ms': 1759918267511, 'update_timestamp_ms': 1759918273172, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'P2HOXCEKN7V6MSVMWCTWYM4U', 'name': 'projects/52933960335/operations/P2HOXCEKN7V6MSVMWCTWYM4U'}
{'state': 'RUNNING', 'description': 'mean_satellite_embeddings_per_polygon', 'priority': 100, 'creation_timestamp_ms': 1759918262736, 'update_timestamp_ms': 1759918273257, 'start_timestamp_ms': 1759918273175, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': '5IVKSMAAJLN6A7X4UDQW6YUB', 'name': 'projects/52933960335/operations/5IVKSMAAJLN6

In [57]:
# Add the elevation model to the map object.
# m.add_ee_layer(reduced, '2025 embeddings')
#
# # Display the map.
# display(m)

Landcover estimate

In [6]:
# Convert to Earth Engine geometry for use in GEE operations (run for about 30 seconds)
increment = 10000  # number of columns to process in each batch
for i in range(0, gdf.shape[0], increment):  # loop
    roi = geemap.geopandas_to_ee(gdf[i:i+increment])
    print(f"Processing rows {i} to {i+increment}...")

    dataset = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2025-03-01', '2025-09-01').filterBounds(roi).mean()
    band_names = dataset.bandNames().getInfo()

    reduced = dataset.reduceRegions(
        collection=roi,
        reducer=ee.Reducer.mean(),
        scale=500  # adjust to your preferred spatial resolution (m)
    )

    selectors = ['grid_id'] + band_names

    # Export the image to a GeoTIFF file.
    task = ee.batch.Export.table.toDrive(
        collection=reduced,
        description='mean_landsat_value_per_polygon',
        fileNamePrefix=f'polygon_landsat_2025_columns_{i}_to_{i+increment}',
        fileFormat='CSV',
        selectors=selectors
    )

    task.start()
    print("✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.")

Processing rows 0 to 10000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 10000 to 20000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 20000 to 30000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 30000 to 40000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 40000 to 50000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 50000 to 60000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 60000 to 70000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 70000 to 80000...
✅ Export started to Google Drive. Check https://code.earthen

Night lights

In [59]:
# Convert to Earth Engine geometry for use in GEE operations (run for about 30 seconds)
increment = 10000  # number of columns to process in each batch
for i in range(0, gdf.shape[0], increment):  # loop
    roi = geemap.geopandas_to_ee(gdf[i:i+increment])
    print(f"Processing rows {i} to {i+increment}...")

    dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filterDate('2025-03-01', '2025-09-01').filterBounds(roi).mean()
    band_names = dataset.bandNames().getInfo()

    reduced = dataset.reduceRegions(
        collection=roi,
        reducer=ee.Reducer.mean(),
        scale=500  # adjust to your preferred spatial resolution (m)
    )

    selectors = ['grid_id'] + band_names

    # Export the image to a GeoTIFF file.
    task = ee.batch.Export.table.toDrive(
        collection=reduced,
        description='mean_night_light_value_per_polygon',
        fileNamePrefix=f'polygon_night_light_2025_columns_{i}_to_{i+increment}',
        fileFormat='CSV',
        selectors=selectors
    )

    task.start()
    print("✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.")

Processing rows 0 to 10000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 10000 to 20000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 20000 to 30000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 30000 to 40000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 40000 to 50000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 50000 to 60000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 60000 to 70000...
✅ Export started to Google Drive. Check https://code.earthengine.google.com/tasks for progress.
Processing rows 70000 to 80000...
✅ Export started to Google Drive. Check https://code.earthen